In [2]:
from pathlib import Path

from src.inspector_git.linker.transformers import GitProjectTransformer
from src.inspector_git.reader.iglog.readers.ig_log_reader import IGLogReader

# 🔹 1. Set the path to a single .iglog file
iglog_path = Path("../../test-input/inspector-git/TestInspectorGitRepo.iglog")

# 🔹 2. Read the IGLog file
with open(iglog_path, "r", encoding="utf-8") as f:
    git_log_dto = IGLogReader().read(f)

# 🔹 3. Transform into a project (compute_annotated_lines=True)
transformer = GitProjectTransformer(
    git_log_dto,
    name=iglog_path.stem,
    compute_annotated_lines=True,
)
project = transformer.transform()

2025-09-15 17:22:02,355 [INFO] src.inspector_git.linker.transformers: Creating GIT project TestInspectorGitRepo
2025-09-15 17:22:02,355 [INFO] src.inspector_git.linker.transformers: Creating commit 1 / 7 (14%)
2025-09-15 17:22:02,359 [INFO] src.inspector_git.linker.transformers: Creating commit 2 / 7 (28%)
2025-09-15 17:22:02,359 [INFO] src.inspector_git.linker.transformers: Creating commit 3 / 7 (42%)
2025-09-15 17:22:02,360 [INFO] src.inspector_git.linker.transformers: Creating commit 4 / 7 (57%)
2025-09-15 17:22:02,361 [INFO] src.inspector_git.linker.transformers: Creating commit 5 / 7 (71%)
2025-09-15 17:22:02,362 [INFO] src.inspector_git.linker.transformers: Creating commit 6 / 7 (85%)
2025-09-15 17:22:02,362 [INFO] src.inspector_git.linker.transformers: Creating commit 7 / 7 (100%)
2025-09-15 17:22:02,363 [INFO] src.inspector_git.linker.transformers: Done creating GIT project TestInspectorGitRepo


In [5]:
# print("Accounts:")
# for acc in project.account_registry.all:
#     print(acc)
#
# print("Commits:")
# for commit in project.commit_registry.all:
#     print(commit)

print("Files:")
for file in project.file_registry.all:
    print(file)
    for change in file.changes:
        print(f"{change}")
        print("\tAnnotated Lines:")
        for idx, line in enumerate(change.annotated_lines, start=1):
            print(f"{idx}\t{line}\t{line.author.git_id.email}")
    print()

Files:
README.md
In 23995e4a76b8a9d6898bffe2ffa7ce8494ed580c : first commit
ChangeType.ADD dev/null->README.md
	Annotated Lines:
1	23995e4a76b8a9d6898bffe2ffa7ce8494ed580c	alexsimiongeorge@gmail.com

random_text.txt
In a55f83ac66177ea82671980c24fb1fd51a7ca295 : added random text file
ChangeType.ADD dev/null->random_text.txt
	Annotated Lines:
1	a55f83ac66177ea82671980c24fb1fd51a7ca295	alexsimiongeorge@gmail.com
2	a55f83ac66177ea82671980c24fb1fd51a7ca295	alexsimiongeorge@gmail.com
In fe38e751c6f9d56dd4cb37420b50fa9b5e43a555 : modified random text file
ChangeType.MODIFY random_text.txt->random_text.txt
	Annotated Lines:
1	fe38e751c6f9d56dd4cb37420b50fa9b5e43a555	alexsimiongeorge@gmail.com
2	a55f83ac66177ea82671980c24fb1fd51a7ca295	alexsimiongeorge@gmail.com
3	fe38e751c6f9d56dd4cb37420b50fa9b5e43a555	alexsimiongeorge@gmail.com

test_file.txt
In 35a4309f9acc856da528221ced720027dc133cad : added test file
ChangeType.ADD dev/null->test_file.txt
	Annotated Lines:
1	35a4309f9acc856da528221ced720

In [1]:
from src.run_blame import parse_blame_file

blame_data = parse_blame_file("../TestInspectorGitRepo_blame.txt")

for file, lines in blame_data.items():
    print(f"File: {file}")
    for entry in lines:
        print(entry)

File: README.md
{'sha': '23995e4', 'email': 'alexsimiongeorge@gmail.com', 'line_index': 1}
File: paralel_file.txt
File: random_text.txt
{'sha': 'fe38e751', 'email': 'alexsimiongeorge@gmail.com', 'line_index': 1}
{'sha': 'a55f83ac', 'email': 'alexsimiongeorge@gmail.com', 'line_index': 2}
{'sha': 'fe38e751', 'email': 'alexsimiongeorge@gmail.com', 'line_index': 3}
File: test_file.txt
{'sha': '35a4309f', 'email': 'alexsimiongeorge@gmail.com', 'line_index': 1}
{'sha': '1e13a769', 'email': 'alexsimiongeorge@gmail.com', 'line_index': 2}


In [17]:
def check_blame(path_blame_file, project):
    match = 0
    total = 0

    project_files = {}
    for file in project.file_registry.all:
        project_files[file.__str__()] = file

    blame_data = parse_blame_file(path_blame_file)

    for file_name, blame_line_entries in blame_data.items():
        git_file = project_files.get(file_name, None)
        if git_file is None:
            print(f"File not found: {file_name}")
            continue
        git_info  = git_file.annotated_lines()

        for entry in blame_line_entries:
            line_index = entry["line_index"]
            sha = entry["sha"]
            email = entry["email"]

            try:
                commit = git_info[line_index - 1]
                if commit.id[:len(sha)] == sha and commit.author.git_id.email == email:
                    match += 1
                else:
                    print(f"Mismatch: {commit.id[:len(sha)]} != {sha} or {commit.author.git_id.email} != {email}")
                total += 1
            except IndexError:
                print(f"Line not found: {line_index} in {file_name}")


    print(f"Match: {match} / Total: {total}")

In [18]:
check_blame("../TestInspectorGitRepo_blame.txt", project)

Match: 6 / Total: 6
